In [1]:

!pip install pyLDAvis

zsh:1: command not found: pip


In [2]:
# -*- coding: utf-8 -*-
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
from gensim.models.coherencemodel import CoherenceModel
import re
import pyLDAvis.gensim_models
import pickle 
import numpy as np, pandas as pd
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt

random_state = 0 

doc_set = """
kawal yuk teman2 pemilu 2024
update hasil quick count pilpres indonesia rabu 17 april 2024 via
sobat polri ayo sukseskan pemilu jaga suara kita hindar dari kecurangan
id wajib pilih cerdas hindari hoaks saat pemilu 2024
kalo mimpi ketemu calon jangan coba dipilih deh kalo udah lanjur pilih jangan coba nyesel seriu
akhbar pilih harga jatuh sindiket gelojoh baca lanjut pa
update data pemilu 2024 17 april 2024 kpu
sobat polri ayo sukseskan pemilu jaga suara kita hindar dari kecurangan
travelling to jakarta from any region make sure your id is updated
instruksi bupati sleman baru guru tinggi selenggara diskusi pemilu masa tenang
pt angkasa pura i persero selenggara pengawasan suara hadap 856
hari banyak 16 juta suara pemilih muda buat perubahan besar di indonesia
pemkot jogja tengah susun ulang atur kait giat ajar ajar kbm tatap muka pemilu
hingga ini 25 dagang malioboro ikut kampanye damai
perempuan lebih mungkin alami efek samping pemilu banding laki
lap kluster baharu kes hari masuk tujuh libat tempat kerja - kp kesihatan
pemilu in karnataka id certificates mandatory for people coming to vote bsybjp
gaiter loops auaaq balaclava bandanas - pemilu
lanjut ica hari mertua alami stroke kondisi rahang miring ki
rektor unp positif terlibat pemilu intensif kampus air tawar
status kini task force operasi patuh pdrm baca lanjut paut
ki ni mangwani yh py paisa lga k awam ko shifa dna ha aur
public speaking butuh sampai informasi cara efisien akurat efektif dengar mampu menangk
suntik 9 juta suara pemilih muda dampak lihat - beritasatu
pengawasan pemilu rawat di wisma atlet mayor per 17 april 2024
bansos pemilu kpk panggil sekjen kemensos dirjen linjamsos - - nasional
tahun the media hotel and towers jadi posko tenaga pengawas pemilu
patut syukur indonesia cepat sukses pemilu - - nasional
catat angka mati hari tinggi akibat hoaks pemilu - - pikir rakyat
budi bicara soal khawatir pemilu - cnbc indonesia
tangan pemilu mungkin besar pengawasan ketat - bisnis
pemilu akibat senjang sosial perempuan makin nyata -
pemilu tahap tujuh - beritasatu
klaim indonesia registrasi calon buat 4 hari sembuh dari kekalahan
pemilu b117 duga masuk garut -
mutasi ganda pemilu temu di india - okezone
3 upaya ojk tangan dampak pemilu -
aktor bollywood aamir khan positif terlibat pemilu -
kampanye pemilu dagang pasar jemput paksa tenaga sehat polisi - inews
wna terlibat pemilu sanur bal pakai id astrazeneca - jawa pos
kampanye pemilu tni al tahap akhir jakarta -
tim grup b piala menpora jalan pengawasan pemilu -
malioboro ingat jalan kampanye damai -
wakil bupati mamuju telat kampanye pemilu - -
modus tipu lowong kerja bayar tengah pemilu - okezone
mark sungkar positif terlibat pemilu - inews
sedia layan pengawasan pemilu klinik pratama makmur jaya ciputat pamulang bintaro isi data pad
kerala
kenya
segera daftar kuota 300 ribu pengawas pemilu
sobat polri ayo sukseskan pemilu jaga suara kita hindar dari kecurangan
program kampanye damai bangsa tan sri dato haji muhyiddin bin haji muhammad yassin perdana menteri malay
innahhh da
lansia jangan takut pilih warga bogor usia 104 tahun sehat pasca coblos
mantap 16 juta suara pemilih muda tiba tanah air pemilu
umbono lingayekethisi i pemilu isesekhona
umum panjang pengawasan sesuai instruksi gubernur diy no 8 instr 2024
panjang pengawasan mikro brebes laku 5 april 2024 pengawasan
seni hibur boleh laksana buah hajat giat sosial masyarakat adala
wakil presiden ma ruf amin imbau masyarakat jalan ibadah rumah lama pemilu langkah harus
italia vs irlandia utara tak gentar sama badai hoaks italy
laksana sambang pos sarlinmas pantai congot kulonprogo personel ditpolair polda diy ajak tugas untu
kapolda diy irjen pol drs asep suhendar hadir tanding pertama piala menpora
kamu malas hanya buang buang waktu kamu tak tahu bagaimana cara raih peluang bahkan pel
survei idm pepet pdip golkar kian moncer jelang pemilu 2024 fraksigolkar
ratus napi nusakambangan positif terlibat pemilu isolasi lapas via
tahan lama pemilu bappenas dorong usaha laku digitalisasi bappenasri
pemilu anak tunggu rekomendasi
sambang kawasan toko jl am sangaji jetis kota yogyakarta personel patroli polsek jetis polresta yogyakar
nu harap semua orang mau ikut pemilu nahdlatululama
prokes 5m dagang unjung pasar bagi masker giat bagai upaya mendi
sobat polri ayo sukseskan pemilu jaga suara kita hindar dari kecurangan
sobat polri ayo sukseskan pemilu jaga suara kita hindar dari kecurangan
sobat polri ayo sukseskan pemilu jaga suara kita hindar dari kecurangan
naik 0 09 usd 1 182 gerak turun dorong tunda amp gelombang 3 m
us index naik 0 03 level 92 55 ini di level tinggi akibat optimisme ak
di-pertuan agong zahir rasa bangga warga pdrm atas korban bagai tugas baris hadap
katno had dukung sukses program pemilu
sobat polri ayo sukseskan pemilu jaga suara kita hindar dari kecurangan
top news koran rakyat merdeka kampanye 1 juta sehari mimpi jokowi
sobat polri ayo sukseskan pemilu jaga suara kita hindar dari kecurangan
id astrazeneca selamat di malaysia
top news koran rakyat merdeka ragu terap protokol sehat jaga pemprov tolak buka 26 tempat karaoke
polri ayo sukseskan pemilu jaga suara a
top news koran rakyat merdeka bule langgar prokes bal dah langsung pulangin negara asal aja bal
top news koran rakyat merdeka soal sekolah tatap muka putus pekan depan satgascovid19
update data hasil pemilu sulawesi barat tanggal 24 maret 2024 total sulbar positif 5358 sembuh 4980 tinggal 1
taat prokes keren jauh kerumun kurang mobilitas polresjogja
update kasus per tanggal 9 april 2024 website
personel jaga pos polairud congot bripka catur widodo s h sama bripka agung h hadir panen raya kete
kisne kaha tha 2024 me shadiyan km hongi ek bar lockdown hone do taara chadne wala hai pemilu
330 perosonil polres ikut pemilu kunjung
patuh protokol sehat 5m polresjogja
tetep gumegrah ojo serah bebarengan lawan hoaks kanthi patuh protokol sehat 5m polresjogja
pakai masker lindung virus pemilu polresjogja
kini task force operasi patuh pdrm baca lanjut paut
salam santun sebar hasil pemilu per tgl 09 apr
19 yongereye ihungabana mu barokotse jenoside mu
sapa masyarakat jumpa tidak lupa himbauan kamtibmas protokol sehat
akibat makin kembang teknologi ini individualisme makin jamur hati tiap orang
sobat jak moga semua segera gilir ikut pemilu ingat atau ses
perintah terap bijak kendali pemilu pulih ekonomi cara imbang
update data pemilu 9 april 2024 kpu
in ai tempi del
kait temu varian baru pemilu e484k satgas minta warga tak panik via
plaza tawar layan check up screening pemilu -
ji ah beri sumbang para siswa dampak pemilu - -
kurang negara dunia rebut suara lawan pemilu - internasional kontan
juang tes pemilu sulit cari murah kualitas -
warga laku ziarah kubur tpu khusus jenazah pemilu -
pegawai positif terlibat pemilu bank sumut cabang tebing tinggi tutup -
pemilu mantan gubernur sumsel mahyudin tinggal dunia -
hari pemilu aceh tingkat baru 42 orang -
transparansi sedia informasi hasil pemilu - viva -
luncur produk baru proteksi suara bantu tangan pemilu - kontan
protokol pemilu maling mobil bikin polisi jadi bahan cemooh -
sekolah tatap muka 10 000 lebih guru tangerang terlibat pemilu - inews
sepeda aman lama pemilu - medcom id
yogyakarta wajib bawa hasil pemilu - kompas tv
luncur program mas








""".split("\n")[1:-1]


stopword = ['pemilu', 'ayo', 'update', 'hasil', 'quick', 'count', 'indonesia', 'rabu', 'april', 'via', 'teman2', 'kita',
    'wajib', 'pilih', 'cerdas', 'hindari', 'saat', 'pemilu', '2024', 'jangan', 'harga', 'data', 'kpu', 'sukseskan',
    'suara', 'hindar', 'kecurangan', 'travelling', 'jakarta', 'region', 'make', 'sure', 'updated', 'instruksi',
    'guru', 'tinggi', 'selenggara', 'masa', 'hingga', 'pemkot', 'tengah', 'susun', 'ulang', 'atur', 'kait', 'giat',
    'ajar', 'kbm', 'ikut', 'kampanye', 'damai']
tokenizer = RegexpTokenizer(r'\w+')


# list for tokenized documents in loop
texts = []

# loop through document list
for i in doc_set:
    
    # clean and tokenize document string
    raw = i.lower()
    raw =  re.sub(r'\b[0-9(.,)+]*\b', '', raw)
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in stopword and len(i) > 2]
    
    
    # add tokens to list
    texts.append(stopped_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]
#nilai K diganti
for k in range(5, 6):
	# generate LDA model
	ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=k, id2word = dictionary, passes=20, iterations=100, alpha=[0.01]*k, eta=[0.01]*len(dictionary.keys()))
	#print(ldamodel.show_topics())
	# Compute Coherence Score
	coherence_model_lda = CoherenceModel(model=ldamodel, texts=texts, dictionary=dictionary, coherence='c_v')
	coherence_lda = coherence_model_lda.get_coherence()
	print(k,ldamodel.log_perplexity(corpus),coherence_lda)  # a measure of how good the model is. lower the better.


def format_topics_sentences(ldamodel=None, corpus=corpus, texts=texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamodel, corpus=corpus, texts=texts)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
print(df_dominant_topic.head(10))



# Sentence Coloring of N Sentences
def topics_per_document(model, corpus, start=0, end=1):
    corpus_sel = corpus[start:end]
    dominant_topics = []
    topic_percentages = []
    for i, corp in enumerate(corpus_sel):
        topic_percs = model[corp]
        dominant_topic = sorted(topic_percs, key = lambda x: x[1], reverse=True)[0][0]
        dominant_topics.append((i, dominant_topic))
        topic_percentages.append(topic_percs)
    return(dominant_topics, topic_percentages)

dominant_topics, topic_percentages = topics_per_document(model=ldamodel, corpus=corpus, start=0, end=4)  

# Distribution of Dominant Topics in Each Document
df = pd.DataFrame(dominant_topics, columns=['Document_Id', 'Dominant_Topic'])

dominant_topic_in_each_doc = df_dominant_topic.groupby('Dominant_Topic').size()
print(dominant_topic_in_each_doc )
df_dominant_topic_in_each_doc = dominant_topic_in_each_doc.to_frame(name='count').reset_index()

# Total Topic Distribution by actual weight
topic_weightage_by_doc = pd.DataFrame([dict(t) for t in topic_percentages])
df_topic_weightage_by_doc = topic_weightage_by_doc.sum().to_frame(name='count').reset_index()

# Top 3 Keywords for each Topic
topic_top3words = [(i, topic) for i, topics in ldamodel.show_topics(formatted=False) 
                                 for j, (topic, wt) in enumerate(topics) if j < 3]

df_top3words_stacked = pd.DataFrame(topic_top3words, columns=['topic_id', 'words'])
df_top3words = df_top3words_stacked.groupby('topic_id').agg(', \n'.join)
df_top3words.reset_index(level=0,inplace=True)

fig, ax1 = plt.subplots(figsize=(10,4))

# Topic Distribution by Dominant Topics
ax1.bar(x='Dominant_Topic', height='count', data=df_dominant_topic_in_each_doc, width=.4, color='firebrick')
ax1.set_xticks(range(df_dominant_topic_in_each_doc.Dominant_Topic.unique().__len__()))
tick_formatter = FuncFormatter(lambda x, pos: 'Topik ' + str(x)+ '\n' + df_top3words.loc[df_top3words.topic_id==x, 'words'].values[0])
ax1.xaxis.set_major_formatter(tick_formatter)
ax1.set_title('Jumlah Tweet Berdasarkan Topik', fontdict=dict(size=10))
ax1.set_ylabel('Jumlah Tweet')
ax1.set_ylim(0, 60)


plt.show()

ImportError: cannot import name 'triu' from 'scipy.linalg' (/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/scipy/linalg/__init__.py)